# Thai Medical QA / RAG — Dataset + Data Engineering Starter (Notebook)

This notebook uses **Thaweewat/thai-med-pack** from Hugging Face and helps you:

- Load + inspect the dataset (Thai medical Q&A style)
- Parse instruction-style text into structured fields: `question`, `answer`
- Do basic data engineering (cleaning, dedupe, privacy/PII-risk scan, safety filtering, split)
- Build a retrieval corpus by chunking answers/contexts
- Export `jsonl` files for model training / RAG indexing

> ⚠️ Note: Some entries may include **sensitive topics** (e.g., sexual health, minors, mental health crises).  
> For an academic MVP, you should **filter** sensitive content and keep the system as **decision-support only**.


In [1]:
# If needed, install deps (uncomment)
# !pip install -U datasets pandas

import re, json, os, random, hashlib
import pandas as pd
from datasets import load_dataset
import tqdm as notebook_tqdm

RANDOM_SEED = 42
random.seed(RANDOM_SEED)


## 1) Load dataset: Thaweewat/thai-med-pack

This dataset is provided as an instruction-style text field (typically one column: `text`).
We'll load the `train` split and parse the `[INST] ... [/INST]` part into question & answer.


In [2]:
ds = load_dataset("Thaweewat/thai-med-pack")
# Typically only 'train' split is provided
df_raw = ds[list(ds.keys())[0]].to_pandas()
df_raw.head()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,text
0,<s>[INST] สวัสดีค่ะ อยากรู้ว่าต้องทำยังไงคะมีอ...
1,<s>[INST] อยากทราบว่ากินยาคลุมฉุกเฉินชนิด 1 เม...
2,<s>[INST] ไม่รู้ว่าใช่ประจำเดือนหรือป่าว แต่เล...
3,<s>[INST] ตอนนี้หนูอายุ13ปีและหนูเป็นนางรำของร...
4,<s>[INST] ตามต้นหัวข้อเลยค่ะ เราเป็นคนที่คิดมา...


## 2) Parse instruction format into structured QA

Expected pattern in `text` (example):
- `<s>[INST] ...question... [/INST] ...answer... </s>`

We will extract:
- `question`
- `answer`

If an entry does not match the pattern, it will be dropped (or kept in `unparsed` for audit).


In [3]:
WS_RE = re.compile(r"\s+")
INST_RE = re.compile(r"\[INST\](.*?)\[/INST\](.*)", re.DOTALL)

def sha1(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8", errors="ignore")).hexdigest()

def clean_text(s) -> str:
    if s is None:
        return ""
    s = str(s).replace("\u00a0", " ")
    s = WS_RE.sub(" ", s).strip()
    return s

def parse_inst(sample: str):
    if not sample:
        return None, None
    m = INST_RE.search(sample)
    if not m:
        return None, None
    q = clean_text(m.group(1))
    a = clean_text(m.group(2))
    # strip common special tokens
    q = q.replace("<s>", "").replace("</s>", "").strip()
    a = a.replace("<s>", "").replace("</s>", "").strip()
    return q, a

# Parse
text_col = "text"
if text_col not in df_raw.columns:
    # Sometimes datasets use 'prompt' or similar; fallback: first column
    text_col = df_raw.columns[0]

parsed = df_raw[text_col].map(parse_inst)
df = pd.DataFrame(parsed.tolist(), columns=["question", "answer"])

# Keep raw for auditing if needed
df["raw_text"] = df_raw[text_col].astype(str).map(lambda x: x[:5000])  # cap length

# Drop unparsed
unparsed = df[df["question"].isna() | df["answer"].isna()].copy()
df = df.dropna(subset=["question", "answer"]).reset_index(drop=True)

# Make stable id
df["id"] = [sha1(f"{q}||{a}||{i}") for i, (q, a) in enumerate(zip(df["question"], df["answer"]))]

df = df[["id", "question", "answer", "raw_text"]]
df.head(), len(df), len(unparsed)


(                                         id  \
 0  cc79cd4e61194d7f30c078bd9f54aa0c94c43f64   
 1  f21051455d5c19c9b7b52096b4fafc3abc7bc88c   
 2  ca64cdd9d62e666f9c72debdbc24945302529c5a   
 3  1e348c10ee823d318858e51c7fddea63a0344ac2   
 4  bd311f898481fe58d118026962b1be27795adbc3   
 
                                             question  \
 0  สวัสดีค่ะ อยากรู้ว่าต้องทำยังไงคะมีอาการคันอวั...   
 1  อยากทราบว่ากินยาคลุมฉุกเฉินชนิด 1 เม็ด แล้วประ...   
 2  ไม่รู้ว่าใช่ประจำเดือนหรือป่าว แต่เลือกมันเป็น...   
 3  ตอนนี้หนูอายุ13ปีและหนูเป็นนางรำของร.ร.แต่หนูร...   
 4  ตามต้นหัวข้อเลยค่ะ เราเป็นคนที่คิดมากอยู่แล้ว ...   
 
                                               answer  \
 0  สวัสดีค่ะ อาการคันอวัยวะเพศหญิง อาจเกิดจาก-รูข...   
 1  สวัสดีค่ะ หลังใข้ยาคุมฉุกเฉินอาจทำให้มีเลือดออ...   
 2  สวัสดีค่ะ หากเลือดที่ออก อยู่ในช่วงวันที่ประจำ...   
 3  สวัสดีค่ะ ฟังจากเหตุการณ์และความคิดของ น่าจะมี...   
 4  สวัสดีค่ะ อาการร้องไห้ง่ายขึ้น มีพฤติกรรมทำร้า...   
 
                      

## 3) Privacy / PDPA + Safety filtering (recommended)

Even if this is a public dataset, the content may include:
- **minors** (age stated)
- sensitive sexual health topics
- mental health crises / self-harm mentions

We will:
1) Do a **conservative PII-risk scan** (email/phone/Thai ID format)  
2) Optionally filter **high-risk sensitive content** using keyword heuristics (editable list)

> This is NOT perfect. Treat it as an engineering safeguard + audit trail.


In [4]:
EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)
PHONE_RE = re.compile(r"\b(?:\+?\d{1,3}[- ]?)?(?:\(?\d{2,3}\)?[- ]?)?\d{3}[- ]?\d{4}\b")
THAI_ID_RE = re.compile(r"\b\d{1}-\d{4}-\d{5}-\d{2}-\d\b")

# Sensitive-topic heuristics (tune for your project)
SENSITIVE_PATTERNS = [
    r"ฆ่าตัวตาย", r"ทำร้ายตัวเอง", r"กรีด", r"ผูกคอ", r"อยากตาย",
    r"ข่มขืน", r"ล่วงละเมิด",
]
SENSITIVE_RE = re.compile("|".join(SENSITIVE_PATTERNS))

def pii_risk_flag(text: str) -> bool:
    if not text:
        return False
    if EMAIL_RE.search(text): return True
    if THAI_ID_RE.search(text): return True
    if PHONE_RE.search(text): return True
    return False

def sensitive_flag(text: str) -> bool:
    if not text:
        return False
    return bool(SENSITIVE_RE.search(text))

flags_pii = df.apply(lambda r: pii_risk_flag(r["question"]) or pii_risk_flag(r["answer"]), axis=1)
flags_sensitive = df.apply(lambda r: sensitive_flag(r["question"]) or sensitive_flag(r["answer"]), axis=1)

df_pii_flagged = df[flags_pii].copy()
df_sensitive_flagged = df[flags_sensitive].copy()

df_kept = df[~(flags_pii | flags_sensitive)].copy().reset_index(drop=True)

len(df), len(df_kept), len(df_pii_flagged), len(df_sensitive_flagged)


(189190, 186172, 358, 2667)

## 4) Dedupe + clean

- Dedupe by `(question, answer)`
- Drop empty rows


In [5]:
def dedupe(df_in: pd.DataFrame) -> pd.DataFrame:
    tmp = df_in.copy()
    tmp["_key"] = (tmp["question"].fillna("") + "||" + tmp["answer"].fillna("")).map(sha1)
    tmp = tmp.drop_duplicates(subset=["_key"]).drop(columns=["_key"]).reset_index(drop=True)
    return tmp

df_clean = dedupe(df_kept)
df_clean = df_clean[(df_clean["question"].str.len() > 0) & (df_clean["answer"].str.len() > 0)].reset_index(drop=True)

df_clean.shape


(186059, 4)

## 5) Split Train/Val/Test

This dataset may not have labels, so we do a random split by default.


In [6]:
def random_split(df_in: pd.DataFrame, train=0.8, val=0.1, test=0.1, seed=RANDOM_SEED):
    assert abs(train + val + test - 1.0) < 1e-9
    idx = list(df_in.index)
    rng = random.Random(seed)
    rng.shuffle(idx)
    n = len(idx)
    n_train = int(n * train)
    n_val = int(n * val)
    train_df = df_in.loc[idx[:n_train]].reset_index(drop=True)
    val_df = df_in.loc[idx[n_train:n_train+n_val]].reset_index(drop=True)
    test_df = df_in.loc[idx[n_train+n_val:]].reset_index(drop=True)
    return train_df, val_df, test_df

train_df, val_df, test_df = random_split(df_clean, 0.8, 0.1, 0.1)

len(train_df), len(val_df), len(test_df)


(148847, 18605, 18607)

## 6) Build retrieval corpus (chunk answers)

For a simple RAG baseline, we can index the **answers** (or combine question+answer).
If you later add external guideline PDFs, replace this with guideline chunking.


In [7]:
def chunk_text(text: str, max_chars: int = 900, overlap: int = 120):
    text = clean_text(text)
    if not text:
        return []
    # sentence-ish split (Thai doesn't use periods consistently; still helps a bit)
    sents = re.split(r"(?<=[.!?。！？])\s+", text)
    chunks, cur = [], ""
    for s in sents:
        if not s:
            continue
        if len(cur) + 1 + len(s) <= max_chars:
            cur = (cur + " " + s).strip()
        else:
            if cur:
                chunks.append(cur)
            cur = s
    if cur:
        chunks.append(cur)

    final = []
    for ch in chunks:
        if len(ch) <= max_chars:
            final.append(ch)
        else:
            start = 0
            while start < len(ch):
                final.append(ch[start:start + max_chars])
                start += max(1, max_chars - overlap)
    return final

def build_corpus(df_in: pd.DataFrame, max_chars=900):
    rows = []
    for _, r in df_in.iterrows():
        doc_id = r["id"]
        # Index answer alone, or (Q+A) if you prefer:
        base = f"Q: {r['question']}\nA: {r['answer']}"
        for j, ch in enumerate(chunk_text(base, max_chars=max_chars, overlap=max(20, int(max_chars*0.15)))):
            rows.append({"doc_id": doc_id, "chunk_id": f"{doc_id}-{j:03d}", "text": ch})
    return pd.DataFrame(rows)

corpus_df = build_corpus(train_df, max_chars=900)
corpus_df.head(), len(corpus_df)


(                                     doc_id  \
 0  cf4f80246e8f4aa4527acde329e708728ef2fb85   
 1  8a119092792fd342caa44cd757208b9b60b0cef2   
 2  8a119092792fd342caa44cd757208b9b60b0cef2   
 3  bf96be8699b1126bff81647bbe4bc236ae49e269   
 4  bf96be8699b1126bff81647bbe4bc236ae49e269   
 
                                        chunk_id  \
 0  cf4f80246e8f4aa4527acde329e708728ef2fb85-000   
 1  8a119092792fd342caa44cd757208b9b60b0cef2-000   
 2  8a119092792fd342caa44cd757208b9b60b0cef2-001   
 3  bf96be8699b1126bff81647bbe4bc236ae49e269-000   
 4  bf96be8699b1126bff81647bbe4bc236ae49e269-001   
 
                                                 text  
 0  Q: ผมโดนเเมวข่วนแมวยังไม่ฉีดยาพึ่งอายุ2เดือนกว...  
 1  Q: เป็นเวลาหนึ่งสัปดาห์แล้วที่ฉันรู้สึกแสบท้อง...  
 2  ราโซล 20 มก., แพนโทพราโซล 40 มก., โอเมพราโซล 2...  
 3  Q: คืิผมอานุแค่14ครับมีตุ่ทที่โคนลิ้นเจ็บไม่เจ...  
 4  เป็นอาการแสดงของโรคติดเชื้อต่างๆ เช่น โรคมือเท...  ,
 225332)

## 7) Export JSONL (+ audit files)

Outputs:
- `qa_train.jsonl`, `qa_val.jsonl`, `qa_test.jsonl`
- `corpus.jsonl`
- `pii_flagged.jsonl` (audit)
- `sensitive_flagged.jsonl` (audit)


In [8]:
def to_jsonl(df_in: pd.DataFrame, path: str):
    with open(path, "w", encoding="utf-8") as f:
        for row in df_in.to_dict(orient="records"):
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

outdir = "out_thai_med_pack"
os.makedirs(outdir, exist_ok=True)

to_jsonl(train_df.drop(columns=["raw_text"]), os.path.join(outdir, "qa_train.jsonl"))
to_jsonl(val_df.drop(columns=["raw_text"]),   os.path.join(outdir, "qa_val.jsonl"))
to_jsonl(test_df.drop(columns=["raw_text"]),  os.path.join(outdir, "qa_test.jsonl"))
to_jsonl(corpus_df, os.path.join(outdir, "corpus.jsonl"))

if len(df_pii_flagged) > 0:
    to_jsonl(df_pii_flagged, os.path.join(outdir, "pii_flagged.jsonl"))
if len(df_sensitive_flagged) > 0:
    to_jsonl(df_sensitive_flagged, os.path.join(outdir, "sensitive_flagged.jsonl"))

stats = {
    "rows_total_parsed": int(len(df)),
    "rows_unparsed": int(len(unparsed)),
    "rows_after_filters": int(len(df_clean)),
    "train": int(len(train_df)),
    "val": int(len(val_df)),
    "test": int(len(test_df)),
    "corpus_chunks_train": int(len(corpus_df)),
    "pii_flagged": int(len(df_pii_flagged)),
    "sensitive_flagged": int(len(df_sensitive_flagged)),
}
with open(os.path.join(outdir, "stats.json"), "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

stats


{'rows_total_parsed': 189190,
 'rows_unparsed': 0,
 'rows_after_filters': 186059,
 'train': 148847,
 'val': 18605,
 'test': 18607,
 'corpus_chunks_train': 225332,
 'pii_flagged': 358,
 'sensitive_flagged': 2667}

## 8) What to write in your proposal/report (copy-paste starter)

**Dataset & Data Description**
- Source: Hugging Face dataset `Thaweewat/thai-med-pack`
- Modality: Thai medical Q&A instruction-style text
- Size: ~189k rows (train split)
- Split: Created locally (80/10/10)

**Privacy / PDPA / Ethics**
- Content may include sensitive health topics and minors; treat as potentially sensitive.
- Mitigations: PII heuristic scan + sensitive-topic filter + audit outputs.
- Risk if wrong: incorrect health guidance -> require disclaimers, escalation rules, and “decision-support only”.

**Data Engineering**
- Parse instruction format, dedupe, drop unparsed/empty.
- Sampling: random split (no labels).
- Assumptions: parsed Q/A reflect intended supervision signal; filtering reduces harmful/sensitive content exposure.


# Feature Engineering

**Feature หลักที่ใช้ (Engineered / Learned)**

* **Combined Q&A Context (Engineered)**: การสร้าง String ชุดใหม่ที่รวมทั้งคำถามและคำตอบเข้าด้วยกัน (รูปแบบ `Q: {question} A: {answer}`) เพื่อให้แต่ละ Chunk มีความหมายที่สมบูรณ์ในตัวเองก่อนนำไปประมวลผล

* **Semantic Chunks (Engineered)**: การแบ่งข้อความเป็นส่วนย่อย (Chunking) ขนาด 900 ตัวอักษร โดยมีส่วนที่ Overlap กัน 120 ตัวอักษร เพื่อรักษาความต่อเนื่องของเนื้อหาทางการแพทย์และป้องกันการขาดหายของบริบทที่รอยต่อ

* **Multilingual MPNet Embeddings (Learned)**: การแปลงข้อความภาษาไทยให้เป็นเวกเตอร์ความหมายขนาด 768 มิติ โดยใช้โมเดล paraphrase-multilingual-mpnet-base-v2 เพื่อใช้ในกระบวนการ Semantic Search

* **Content-Based Stable ID (Engineered)**: การสร้าง ID เฉพาะตัวด้วยการทำ SHA1 Hash จากเนื้อหา Q&A เพื่อใช้ระบุตัวตนของข้อมูลและป้องกันความซ้ำซ้อนในฐานข้อมูลเวกเตอร์

**เหตุผลในการเลือก Feature**

* **Semantic Mapping**: เลือกใช้ paraphrase-multilingual-mpnet-base-v2 เนื่องจากเป็นโมเดลที่ถูกฝึกมาเพื่อเน้นเรื่องการเปรียบเทียบความหมายของประโยค และรองรับภาษาไทยได้ดี ทำให้สามารถดึงข้อมูลที่เกี่ยวข้องได้แม้ผู้ใช้จะใช้คำถามที่ไม่ตรงกับ Keyword ในฐานข้อมูล

* **Contextual Integrity**: การทำ Chunking พร้อม Overlap ช่วยให้ระบบ RAG สามารถเข้าถึงข้อมูลที่เป็นคำแนะนำทางการแพทย์ได้อย่างต่อเนื่องและลดความผิดพลาดในการสร้างคำตอบ

**Feature ที่ตั้งใจไม่ใช้ (ถ้ามี)**

* **Raw Instruction Tokens**: เช่น `<s>`, `[INST]`, `[/INST]`, `</s> `จะถูกคัดออกเนื่องจากเป็นสัญลักษณ์ควบคุมทางเทคนิคที่ไม่ส่งผลต่อความหมายในเชิงการค้นหา

* **High-Risk Sensitive Mentions**: เนื้อหาที่เกี่ยวข้องกับภาวะวิกฤต เช่น การทำร้ายตัวเอง หรือการฆ่าตัวตาย ที่ถูก Flag ไว้ในขั้นตอนการทำความสะอาดข้อมูล จะไม่ถูกนำมาสร้างเป็น Feature เพื่อความปลอดภัยตามหลักจริยธรรมทางการแพทย์

* **PII (Personally Identifiable Information)**: ข้อมูลส่วนบุคคล เช่น หมายเลขโทรศัพท์ หรือเลขประจำตัวประชาชน จะถูกคัดออกเพื่อปฏิบัติตามหลักความเป็นส่วนตัวของข้อมูลสุขภาพ (PDPA)

Text Embedding using `paraphrase-multilingual-mpnet-base-v2`

In [ ]:
# !pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_embeddings(df_corpus):
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)

    sentences = df_corpus['text'].tolist()

    embeddings = model.encode(
        sentences,
        show_progress_bar=True,
        convert_to_numpy=True,
        batch_size=32
    )

    return embeddings

corpus_embeddings = generate_embeddings(corpus_df)
np.save("out_thai_med_pack/corpus_embeddings.npy", corpus_embeddings)

# Model Section

In [ ]:
# !pip install -U accelerate sentence-transformers
# !pip install -q -U faiss-cpu
# !pip install openai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.2.2
    Uninstalling sentence-transformers-5.2.2:
      Successfully uninstalled sentence-transformers-5.2.2


## load embedding model

In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np
embed_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## RAG Knowledge Ingestion

In [16]:
import faiss
embeddings = np.load("out_thai_med_pack/corpus_embeddings.npy").astype('float32')

d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embeddings)

medical_kb = corpus_df.to_dict('records')

In [19]:
print(medical_kb[0])

{'doc_id': 'cf4f80246e8f4aa4527acde329e708728ef2fb85', 'chunk_id': 'cf4f80246e8f4aa4527acde329e708728ef2fb85-000', 'text': 'Q: ผมโดนเเมวข่วนแมวยังไม่ฉีดยาพึ่งอายุ2เดือนกว่าๆแบบนี้จะเป็นไรไหมครับแต่ผมเคยฉีดวัคซีนกันพิษสุนัขบ้ามาแล้วครับครบ5เข้มแบบนี้ผมต้องไปฉีดยาไหมครับรู้สึกไม่สบายใจ A: สวัสดีค่ะ โรคพิษสุนัขบ้านั้นเกิดเมื่อถูกสัตว์เลี้ยงลูกด้วยนมเช่น สุนัข แมว หนูกัด หรือไปสัมผัสสารคัดหลั่งของมันเข้าไปตรงแผลหรือเยื่อบุเช่น ในปาก โดยตรง โรคพิษสุนัขบ้าป้องกันได้ด้วยการฉีดวัคซีน 5 เข็ม และมีการกระตุ้นวัคซีนเมื่อถูกกัดซ้ำจากที่กล่าวมานั้น ถ้าเคยรับวัคซีนป้องกันพิษสุนัขบ้ามาครบ 5 เข็มแต่นานเกิน 6 เดือนมาแล้ว ควรจะไปกระตุ้น แม้ว่าจริงๆแล้วหากได้รับวัคซีนครบ 5 เข็มภูมิคุ้มกันจะอยู่สูงได้ประมาณ 1-2 ปีก็ตาม การกระตุ้นวัคซีนนั้น ถ้าถูกกัดมาภายใน 6 เดือน จะกระตุ้น 1 เข็ม ถ้าเกิน 6 เดือนกระตุ้น 2 เข็มห่างกัน 3 วัน'}


## Candidate model setup
- baseline model : gemini-3-flash-preview
- candidate 1 : Gemma-SEA-LION-v4-27B-IT
- candidate 2 : Kimi-K2.5

In [ ]:
from google.colab import userdata
from openai import OpenAI
import google.generativeai as genai
import os

# candidate model list
LLM_CHOICES = {
    "gemini": { # baseline model
        "type": "gemini",
        "api_key": userdata.get("gemini_api"), # get env in collab
        "model": "gemini-3-flash-preview"
    },
    "sealions": { # candidate 1
        "type": "openai",
        "api_key": userdata.get('sealion_api'),
        "base_url": "https://api.sea-lion.ai/v1",
        "model": "aisingapore/Gemma-SEA-LION-v4-27B-IT"
    },
    "kimi": { # candidate 2
        "type": "openai",
        "api_key": userdata.get('openrouter_api'),
        "base_url": "https://openrouter.ai/api/v1",
        "model": "moonshotai/Kimi-K2.5"
    },
}
def get_llm_client(provider_name):
    config = LLM_CHOICES[provider_name]

    if config["type"] == "openai":
        client = OpenAI(
            api_key=config["api_key"],
            base_url=config["base_url"]
        )
        return client, config["model"], "openai"

    if config["type"] == "gemini":
        genai.configure(api_key=config["api_key"])
        model = genai.GenerativeModel(config["model"])
        return model, config["model"], "gemini"



/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


## Setup model context

In [26]:
def medical_rag_system(user_query, provider="kimi", temperature = 0.1):
    # Step: Search
    query_embedding = embed_model.encode([user_query], normalize_embeddings=True)
    _, indices = index.search(np.array(query_embedding).astype('float32'), k=1)

    match = medical_kb[indices[0][0]]


# FILL IN SYSTEM PROMPT HERE
    messages = [
        {
            "role": "system",
            "content": """
        คุณคือผู้ช่วยคัดกรองอาการผู้ป่วย
        ใช้เฉพาะข้อมูลอ้างอิงที่ให้
        ต้องตอบเป็น 3 ส่วน:
        1. การประเมินอาการ
        2. ระดับความรุนแรง
        3. แผนกที่ควรไป
        สรุปการแนะนำ
        ตอบภาษาเดียวกับคำถาม
        ห้ามทวนคำสั่ง
        """
        },
        {
    "role": "user",
    "content": f"""
ข้อมูลอ้างอิง:
{match['text']}

คำถาม:
{user_query}
"""
        }
    ]
    client, model_name, provider_type = get_llm_client(provider)
    print("using model: " + model_name)
    if provider_type == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=temperature,
            max_tokens= 2048
        )
        answer = response.choices[0].message.content

    elif provider_type == "gemini":
        # Gemini expects single prompt string
        full_prompt = "\n".join([m["content"] for m in messages])

        response = client.generate_content(
            full_prompt,
            generation_config={
                "temperature": temperature,
                "max_output_tokens": 2048
            }
        )
        answer = response.text

    disclaimer = "\n\n*หมายเหตุ: นี่คือการคัดกรองและการแนะนำเบื้องต้น ไม่ใช่การวินิจฉัยทางการแพทย์*"
    return answer + disclaimer


## Q&A Implementation

In [29]:
query = "เจ็บหน้าอกมาก หายใจลำบาก"
print(f"ผลลัพธ์การคัดกรอง:\n{medical_rag_system(query,provider="gemini")}")

{'doc_id': '86c7c3cb40e11003486a9efa833d5fd9206a967b', 'chunk_id': '86c7c3cb40e11003486a9efa833d5fd9206a967b-000', 'text': 'Q: ฉันหายใจลำบากและแน่นหน้าอก ฉันยังรู้สึกหายใจติดขัด A: จากอาการของคุณ ดูเหมือนว่าคุณเป็นโรคหลอดลมอักเสบเฉียบพลัน'}
Q: ฉันหายใจลำบากและแน่นหน้าอก ฉันยังรู้สึกหายใจติดขัด A: จากอาการของคุณ ดูเหมือนว่าคุณเป็นโรคหลอดลมอักเสบเฉียบพลัน
using model: gemini-3-flash-preview
ผลลัพธ์การคัดกรอง:
1. **การประเมินอาการ**: จากอาการเจ็บหน้าอกมากและหายใจลำบาก ดูเหมือนว่าคุณเป็นโรคหลอดลมอักเสบเฉียบพลัน
2. **ระดับความรุนแรง**: รุนแรง
3. **แผนกที่ควรไป**: แผนกฉุกเฉิน หรือ แผนกอายุรกรรม

**สรุปการแนะนำ**: เนื่องจากคุณมีอาการเจ็บหน้าอกและหายใจลำบาก ซึ่งเข้าข่ายอาการของโรคหลอดลมอักเสบเฉียบพลันตามข้อมูลอ้างอิง และมีความรุนแรงของอาการมาก ควรไปพบแพทย์ทันทีเพื่อรับการตรวจวินิจฉัยและรักษาอย่างเร่งด่วน

*หมายเหตุ: นี่คือการคัดกรองและการแนะนำเบื้องต้น ไม่ใช่การวินิจฉัยทางการแพทย์*


# **What to fill in proposal**

## LLM Model Selection

## 1. Baseline Model: Gemini 3 Flash
**Gemini 3 Flash** serves as the primary benchmark for this study due to its balance of speed and advanced reasoning.

### Rationale for Selection
* **Instruction Following:** Demonstrates high proficiency in complex reasoning and adhering to strict prompt constraints.
* **Multilingual Support:** Native processing capabilities for Southeast Asian languages, specifically **Thai**.
* **Task Stability:** Provides consistent performance across structured tasks such as classification, extraction, and summarization.
* **Documentation:** Widely adopted and well-documented, making it an ideal reference point for comparative analysis.

### Comparition metric
![Artificial Analysis Intelligence Index](https://artificialanalysis.ai/img/articles/gemini-3-flash-everything-you-need-to-know/Artificial_Analysis_Intelligence_Index_%2816_Dec_25%29.png)

---
## 2. Candidate Models

### A. SEA-LION (Gemma-based 27B Instruction-tuned)
A model specifically architecturalized for the Southeast Asian (SEA) region.

* **Key Strengths:**  Optimized for regional linguistic nuances.
    * Strong alignment with local medical and cultural expressions.
    * Open-weight architecture, providing greater deployment flexibility and private hosting capabilities.
* **Comparison to Baseline:**  **Optimization** Regionally focused vs. Gemini’s global training.
    * **Reasoning:** May trade off some general reasoning for superior local context.

### B. Kimi 2.5
A high-performance model known for its efficiency in handling large datasets.

* **Key Strengths:**
    * Exceptional **long-context handling** for processing lengthy documents.
    * Strong structured output consistency.
    * Competitive cost-to-performance ratio in production environments.
* **Comparison to Baseline:**
    * **Architecture:** Offers a different training approach compared to Google’s Gemini series.
    * **Context:** Prioritizes maintaining coherence over massive input tokens.
  ![kimi_metric](https://miro.medium.com/1*Ycy0aWssByBlhf0pb88CWg.png)

---

## 3. Comparative Summary

| Feature | Gemini 3 Flash (Baseline) | SEA-LION (27B) | Kimi 2.5 |
| :--- | :--- | :--- | :--- |
| **Optimization** | Global / General | Southeast Asian Regional | Long-Context / Efficiency


---

### **Embedding Model** (Candidate)
**1. bge-m3 :** vector embedding model
![bge-m3](https://scontent.fbkk12-5.fna.fbcdn.net/v/t39.30808-6/504256971_3984009171837940_2984735354200405157_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=aa7b47&_nc_ohc=GrsYXizIJ2MQ7kNvwExDhEt&_nc_oc=AdmT3VCJlAgoYPWfAQemWXXsGkjvCu6BzgA1XwU5cW_9oXT36Lxmsr9_seJD8gv8q5A&_nc_zt=23&_nc_ht=scontent.fbkk12-5.fna&_nc_gid=7_oZaUIZbMQJeefqnBHPkA&oh=00_AfuxZNLD1XXfXyoSGVaizHACVPIaUWjCbVxvPA9zFXLKlQ&oe=699A4510)
reference: https://huggingface.co/spaces/panuthept/thai_sentence_embedding_benchmark

BGE-M3 is specifically trained to handle over 100 languages. In the context of our's project, it excels at Semantic Mapping—understanding that a patient’s casual description of a symptom (e.g., "ปวดจี๊ดๆ ที่อก") carries the same semantic weight as formal medical terms in our's database (e.g., "Chest pain" or "Angina").

